# グラフェンの変形をできるだけおこさなくする。

(2024-11-05)

graphene-studio で生成した炭素配置をできるだけ維持するように、分子内の拘束条件(constraint)を減らすかわりに、原子を正しい位置に束縛(restraint)する。

## 注意

水分子モデルが TIP4P になっている。


## 1. 水和させる

graphene-studio で生成した構造の 1 つを選び、ファイル`graphene.gro`にコピーする。

その構造を水和させる。あらかじめ、セルの大きさは z 方向に 2 倍になっている。


In [1]:
! cp diamond_1.itp graphene.itp
! cp diamond_1.gro graphene.gro
! gmx solvate -cs tip4p -cp graphene.gro -o solvated.gro -p solvated-fullfix.top 


                 :-) GROMACS - gmx solvate, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx solvate -cs tip4p -cp graphene.gro -o solvated.gro -p solvated-fullfix.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be di

## 2. 炭素原子をゆるく固定する。

(手順は`HMTA/`からもってきた。)

炭素を定位置に固定(restraint)する外場は`graphene-restraints.itp`ファイルに書く。こんな感じの拘束条件を原子の個数だけ書きならべる。

```shell
[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz
   1    1       100000       100000       100000
```

原子の個数だけバネが必要なので長大なデータになる。自動生成させるのが良い。
以下のコードで、position_restraints を自動生成する。


In [ ]:
from graphenestudio.gromacs import read_gro

# 原子数を読みとるだけのためにgraphene.groを読みこんでいる。
with open("graphene.gro") as f:
    for frame in read_gro(f):
        Natom = frame.residue_id.shape[0]
        break

# 原子の個数だけバネをつける。
with open("graphene-restraints.itp", "w") as f:
    print("""[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz""", file=f)
    for i in range(Natom):
        print(f"{i+1} 1 10000 10000 10000", file=f)



あるいはコマンドラインで


In [2]:
! python restrain.py < graphene.gro > graphene-restraints.itp

## 3. MD の準備

- `-r`オプションで固定先の座標を指定する。錨をおろす位置は継続計算でも変更してはいけない。
- シミュレーションの条件は`initial-solvated-fullfix.mdp`に書かれている。ここでは、圧力制御を`berendsen`とし、`semiisotropic`な変形を許す。
- `solvated-fullfix.top`の中で、上で作った`graphene-restraints.itp`が読みこまれることで、どの原子を固定するかが指示されている。
- 原子数があわないというエラーが出る場合は、`solvated-fullfix.top`ファイルに書きこまれた水和水分子数が正しいかどうかを確認する。
- 水分子モデルを変更する場合にも`solvated-fullfix.top`を編集する。


In [4]:
! gmx grompp -maxwarn 2 -f initial-solvated-fullfix.mdp -p solvated-fullfix.top -o 00001.tpr -c solvated.gro -r solvated.gro

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx grompp -maxwarn 2 -f initial-solvated-fullfix.mdp -p solvated-fullfix.top -o 00001.tpr -c solvated.gro -r solvated.gro

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'cpp'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'

WARNING 1 [file initial-solvated-fullfix.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to -118327617

Generated 332520 of the 332520 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 1

Generated 332520 of the 332520 1-4 parameter combina

## 4. 初回の MD 実行


In [63]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00001 -c 00001-last.gro

                  :-) GROMACS - gmx mdrun, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx mdrun -notunepme -nt 8 -deffnm 00001 -c 00001-last.gro


Back Off! I just backed up 00001.log to ./#00001.log.9#
Reading file 00001.tpr, VERSION 2024.2-Homebrew (single precision)
Changing nstlist from 10 to 100, rlist from 0.9 to 0.904

NOTE: Periodic molecules are present in this system. Because of this, the domain decomposition algorithm cannot easily determine the minimum cell size that it requires for treating bonded interactions. Instead, domain decomposition will assume that half the non-bonded cut-off will be a suitable lower bound.
Using 1 MPI thread
Using 8 OpenMP threads 


Back Off! I just backed up 00001.trr to ./#00001.trr.9#

Back Off! I just backed up 00001.edr to ./#00001.edr.9#
starting mdrun 'solvated graphen

In [ ]:
! make 00001-0.gro

make: `00001-0.gro' is up to date.


## 5. 継続計算

継続する場合も、炭素原子固定位置を指示する部分はそのまま使う。

- `continue-solvated-fullfix.mdp`での変更点は
  1. 温度を 270 K に。
  2. extended_simulation に関する設定
     - `gen_vel                  = no`
     - `continuation             = yes`
  3. dt をまともな数字(0.001 ps)に。
  4. 圧力を 1 気圧に設定。

`Nose-Hoover`では異常に分子運動が抑えられてしまうので、やむをえず`v-rescale`を継続使用している。


`diff`コマンドで設定の違いを比較する。


In [79]:
! diff initial-solvated-fullfix.mdp continue-solvated-fullfix.mdp

8c8
< dt                       = 0.0001     ; MD計算のΔt (ps)。
---
> dt                       = 0.001     ; MD計算のΔt (ps)。
42c42
< ref_t                    = 300.0       ; 系の温度。単位はK。
---
> ref_t                    = 270.0       ; 系の温度。単位はK。
49c49
< ref_p                    = 1    1000     ; 系の圧力。単位はbar。Pcoupl = no の場合は無効。
---
> ref_p                    = 1    1    ; 系の圧力。単位はbar。Pcoupl = no の場合は無効。
52,53c52,53
< gen_vel                  = yes
< continuation             = no
---
> gen_vel                  = no
> continuation             = yes


In [77]:
! gmx grompp -maxwarn 2 -f continue-solvated-fullfix.mdp -p solvated-fullfix.top -o 00002.tpr -c 00001-last.gro -r solvated.gro

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx grompp -maxwarn 2 -f continue-solvated-fullfix.mdp -p solvated-fullfix.top -o 00002.tpr -c 00001-last.gro -r solvated.gro

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'cpp'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'

WARNING 1 [file continue-solvated-fullfix.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to -281020425

Generated 332520 of the 332520 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 1

Generated 332520 of the 332520 1-4 parameter com

In [78]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00002 -c 00002-last.gro

                  :-) GROMACS - gmx mdrun, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx mdrun -notunepme -nt 8 -deffnm 00002 -c 00002-last.gro


Back Off! I just backed up 00002.log to ./#00002.log.11#
Reading file 00002.tpr, VERSION 2024.2-Homebrew (single precision)
Changing nstlist from 10 to 80, rlist from 0.901 to 1.007

NOTE: Periodic molecules are present in this system. Because of this, the domain decomposition algorithm cannot easily determine the minimum cell size that it requires for treating bonded interactions. Instead, domain decomposition will assume that half the non-bonded cut-off will be a suitable lower bound.
Using 1 MPI thread
Using 8 OpenMP threads 


Back Off! I just backed up 00002.trr to ./#00002.trr.11#

Back Off! I just backed up 00002.edr to ./#00002.edr.11#
starting mdrun 'solvated gra